Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [902]:
from random import choices
from dataclasses import dataclass
from copy import copy
from random import randint, random, choice, uniform
import lab9_lib

fitness = lab9_lib.make_problem(2)

for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")


print(fitness.calls)

In [903]:
POPULATION_SIZE = 60
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 10
MUTATION_PROBABILITY = .15

SEQUENCE_LENGTH = 1000

In [904]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

### Selection

In [905]:
class Tournament():
    def __init__(self, size):
        self.size = size

    def select(self, pop):
        pool = [choice(pop) for _ in range(self.size)]
        champion = max(pool, key=lambda i: i.fitness)
        return champion
    
class TournamentEpsilon():
    def __init__(self, size, scheduler):
        self.size = size
        self.schedular = scheduler
    
    def select(self, pop):
        if random() > self.schedular.value():
            pool = [choice(pop) for _ in range(self.size)]
            champion = max(pool, key=lambda i: i.fitness)
            return champion
        else:
            return choice(pop)
        
def tweak(ind: Individual):
    obj = copy(ind)
    pos = randint(0, len(obj.genotype) - 1)
    pos2 = randint(0, len(obj.genotype) - 1)


    obj.genotype[pos] = not obj.genotype[pos]
    obj.genotype[pos2] = not obj.genotype[pos2]

    obj.fitness = None

    return obj     

def hill_climbing(ind: Individual, n_iter = 10):
    s = copy(ind)

    for _ in range(n_iter):
        r = tweak(s)
        if fitness(r.genotype) > fitness(s.genotype):
            s = r
    
    return s

### Mutation

In [906]:
class Scheduler():
    def __init__(self, alpha, decay_rate = 0):
        self.rate = decay_rate
        self.current_value = alpha
    
    def mutation_prob():
        pass

class ConstantScheduler(Scheduler):
    def value(self):
        return self.current_value

class ExponentialDecayScheduler(Scheduler):
    def value(self):
        return_value = self.current_value
        self.current_value = self.current_value * (1 - self.rate)
       
        return return_value
    
class BitFlip():
    def __init__(self, sequence_length, scheduler):
        self.sequence_length = sequence_length
        self.scheduler = scheduler
    
    def mutate(self, ind: Individual) -> Individual:
        offspring = copy(ind)

        if random() < self.scheduler.value():
            pos = randint(0, self.sequence_length-1)
            #offspring.genotype[pos] = not offspring.genotype[pos]
            offspring.genotype[pos] = uniform(0,1)

            offspring.fitness = None
        
        return offspring



### Recombination

In [907]:
class OnePointCrossover():
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length

    def recombine(self,ind1: Individual, ind2: Individual) -> Individual:
        cut_point = randint(0, self.sequence_length-1)
        offspring1 = Individual(fitness=None,
                            genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
        offspring2 = Individual(fitness=None,
                            genotype=ind2.genotype[:cut_point] + ind1.genotype[cut_point:])

        assert len(offspring1.genotype) == self.sequence_length
        assert len(offspring1.genotype) == self.sequence_length

        return offspring1, offspring2

class UniformCrossover():
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length
    
    def recombine(self, ind1: Individual, ind2: Individual) -> Individual:
        rvector1 = [random() for _ in range(self.sequence_length)]
        rvector2 = [random() for _ in range(self.sequence_length)]

        g1 = [p1 if r < 0.5 else p2 for p1, p2, r in zip(ind1.genotype, ind2.genotype, rvector1)]
        g2 = [p1 if r < 0.5 else p2 for p1, p2, r in zip(ind1.genotype, ind2.genotype, rvector2)]

        o1 = Individual(fitness = None, genotype=g1)
        o2 = Individual(fitness = None, genotype=g2)

        return o1, o2



In [908]:
class EASimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []

        for _ in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                offspring.append(self.mutate(o1))
                offspring.append(self.mutate(o2))

            #Evaluate offsprings
            for i in offspring:
                i.fitness = self.fitness_eval(i.genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

            if self.population[0].fitness == 1:
                break

        return record
    
class HybridSimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []

        for _ in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                offspring.append(self.mutate(o1))
                offspring.append(self.mutate(o2))

            #Evaluate offsprings
            for i in range(len(offspring)):
                offspring[i] = hill_climbing(offspring[i])
                offspring[i].fitness = self.fitness_eval(offspring[i].genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

        return record
    

class AdaptiveSimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []
        explorative_control = 1
        best = 0
        init_pop = self.population

        for g in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                for _ in range(explorative_control):
                    o1 = self.mutate(o1)
                    o2 = self.mutate(o2)

                offspring.append(o1)
                offspring.append(o2)

            #Evaluate offsprings
            for i in offspring:
                i.fitness = self.fitness_eval(i.genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

            if self.population[0].fitness > best:
                best = self.population[0].fitness
                explorative_control = 1

            elif self.population[0].fitness == best:
                explorative_control = explorative_control + 1

                if explorative_control > 10:
                    new_pop = []

                    for _ in range(self.population_size):
                        if random() > 0.5:
                            new_pop.append(choice(self.population))
                        else:
                            new_pop.append(choice(init_pop))

                    self.population = new_pop
            else:
                explorative_control = explorative_control * 2
                
            print(f'generation:{g}_explorecontrol:{explorative_control}')
            
            if self.population[0].fitness == 1:
                break

        return record
    

class GreedyAdaptiveSimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []
        explorative_control = 1
        best = 0
        init_pop = self.population

        for g in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                for _ in range(explorative_control):
                    o1 = self.mutate(o1)
                    o2 = self.mutate(o2)

                offspring.append(o1)
                offspring.append(o2)

            #Evaluate offsprings
            for i in offspring:
                i.fitness = self.fitness_eval(i.genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

            if self.population[0].fitness > best:
                best = self.population[0].fitness
                explorative_control = 1

            elif self.population[0].fitness == best:
                explorative_control = explorative_control + 1

                if explorative_control > 10:
                    new_pop = []

                    for _ in range(self.population_size):
                        if random() > 0.7:
                            new_pop.append(choice(self.population[:explorative_control]))
                        else:
                            new_pop.append(choice(init_pop))

                    self.population = new_pop
            else:
                explorative_control = explorative_control //2 + 1
                
            print(f'generation:{g}_explorecontrol:{explorative_control}')
            
            if self.population[0].fitness == 1:
                break

        return record


        

### Initialize Population

In [909]:
population = [
    Individual(
        genotype=[choice((True, False)) for _ in range(SEQUENCE_LENGTH)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]



### Experiment

#### instance 1

In [910]:
fitness = lab9_lib.make_problem(1)

In [911]:
for i in population:
    i.fitness = fitness(i.genotype)

In [912]:
#selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.00001)
selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE , scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.0)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH)

sim = AdaptiveSimulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim.set_inital_pop(population)

record = sim.simulate()

generation:0_explorecontrol:1
generation:1_explorecontrol:2
generation:2_explorecontrol:1
generation:3_explorecontrol:1
generation:4_explorecontrol:2
generation:5_explorecontrol:1
generation:6_explorecontrol:1
generation:7_explorecontrol:1
generation:8_explorecontrol:1
generation:9_explorecontrol:1
generation:10_explorecontrol:1


generation:11_explorecontrol:2
generation:12_explorecontrol:1
generation:13_explorecontrol:1
generation:14_explorecontrol:2
generation:15_explorecontrol:1
generation:16_explorecontrol:1
generation:17_explorecontrol:1
generation:18_explorecontrol:1
generation:19_explorecontrol:1
generation:20_explorecontrol:1
generation:21_explorecontrol:1
generation:22_explorecontrol:1
generation:23_explorecontrol:1
generation:24_explorecontrol:1
generation:25_explorecontrol:1
generation:26_explorecontrol:1
generation:27_explorecontrol:1
generation:28_explorecontrol:1
generation:29_explorecontrol:1
generation:30_explorecontrol:1
generation:31_explorecontrol:1
generation:32_explorecontrol:2
generation:33_explorecontrol:1
generation:34_explorecontrol:1
generation:35_explorecontrol:1
generation:36_explorecontrol:1
generation:37_explorecontrol:1
generation:38_explorecontrol:1
generation:39_explorecontrol:2
generation:40_explorecontrol:1
generation:41_explorecontrol:1
generation:42_explorecontrol:1
generati

In [913]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 1.0
num calls: 13080


#### instance 2

In [914]:
fitness = lab9_lib.make_problem(2)

In [915]:
for i in population:
    i.fitness = fitness(i.genotype)

In [916]:
#selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.00001)
selection_scheduler = ConstantScheduler(alpha = 0)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE, scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH)

sim = AdaptiveSimulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim.set_inital_pop(population)

record = sim.simulate()

generation:0_explorecontrol:1
generation:1_explorecontrol:1
generation:2_explorecontrol:1
generation:3_explorecontrol:1
generation:4_explorecontrol:2
generation:5_explorecontrol:1
generation:6_explorecontrol:1
generation:7_explorecontrol:1
generation:8_explorecontrol:1
generation:9_explorecontrol:2
generation:10_explorecontrol:1
generation:11_explorecontrol:2
generation:12_explorecontrol:1
generation:13_explorecontrol:2
generation:14_explorecontrol:1
generation:15_explorecontrol:1
generation:16_explorecontrol:1
generation:17_explorecontrol:2
generation:18_explorecontrol:3
generation:19_explorecontrol:1
generation:20_explorecontrol:1
generation:21_explorecontrol:1
generation:22_explorecontrol:1
generation:23_explorecontrol:1
generation:24_explorecontrol:1
generation:25_explorecontrol:2
generation:26_explorecontrol:1
generation:27_explorecontrol:2
generation:28_explorecontrol:1
generation:29_explorecontrol:2
generation:30_explorecontrol:1
generation:31_explorecontrol:1
generation:32_expl

In [917]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 1.0
num calls: 33360


#### instance 3

In [926]:
fitness = lab9_lib.make_problem(5)

In [927]:
for i in population:
    i.fitness = fitness(i.genotype)

In [928]:
#selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.000001)
selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE, scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.0)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH)

sim = AdaptiveSimulation(population_size=POPULATION_SIZE , offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim.set_inital_pop(population)

record = sim.simulate()

generation:0_explorecontrol:1
generation:1_explorecontrol:1
generation:2_explorecontrol:2
generation:3_explorecontrol:3
generation:4_explorecontrol:1
generation:5_explorecontrol:2
generation:6_explorecontrol:3
generation:7_explorecontrol:4
generation:8_explorecontrol:5
generation:9_explorecontrol:6
generation:10_explorecontrol:7
generation:11_explorecontrol:8
generation:12_explorecontrol:9
generation:13_explorecontrol:10
generation:14_explorecontrol:11
generation:15_explorecontrol:12
generation:16_explorecontrol:24
generation:17_explorecontrol:48
generation:18_explorecontrol:96
generation:19_explorecontrol:1
generation:20_explorecontrol:2
generation:21_explorecontrol:3
generation:22_explorecontrol:4
generation:23_explorecontrol:5
generation:24_explorecontrol:6
generation:25_explorecontrol:7
generation:26_explorecontrol:1
generation:27_explorecontrol:2
generation:28_explorecontrol:3
generation:29_explorecontrol:4
generation:30_explorecontrol:5
generation:31_explorecontrol:6
generation:3

In [929]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 1.0
num calls: 7500


#### instance 4

In [930]:
fitness = lab9_lib.make_problem(10)

In [931]:
for i in population:
    i.fitness = fitness(i.genotype)

In [932]:
#selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.000001)
selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE, scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.0)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH)

sim = AdaptiveSimulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim.set_inital_pop(population)

record = sim.simulate()

generation:0_explorecontrol:1
generation:1_explorecontrol:2
generation:2_explorecontrol:3
generation:3_explorecontrol:4
generation:4_explorecontrol:1
generation:5_explorecontrol:2
generation:6_explorecontrol:1
generation:7_explorecontrol:2
generation:8_explorecontrol:3
generation:9_explorecontrol:1
generation:10_explorecontrol:1
generation:11_explorecontrol:2
generation:12_explorecontrol:3
generation:13_explorecontrol:4
generation:14_explorecontrol:1
generation:15_explorecontrol:2
generation:16_explorecontrol:3
generation:17_explorecontrol:4
generation:18_explorecontrol:5
generation:19_explorecontrol:1
generation:20_explorecontrol:2
generation:21_explorecontrol:3
generation:22_explorecontrol:4
generation:23_explorecontrol:5
generation:24_explorecontrol:6
generation:25_explorecontrol:7
generation:26_explorecontrol:8
generation:27_explorecontrol:9
generation:28_explorecontrol:10
generation:29_explorecontrol:11
generation:30_explorecontrol:1
generation:31_explorecontrol:2
generation:32_ex

In [933]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 1.0
num calls: 9240
